In [1]:
import pymongo
import charts
import datetime

Server running in the folder /Users/Guoyin/Documents/Course/web_crawler/Plan-for-combating-master/week3/week3_homework at 127.0.0.1:64644


In [2]:
client = pymongo.MongoClient('localhost',27017)
test = client['test']
item_info = test['item_info']

In [4]:
for item in item_info.find().limit(10):
    print(item)

{'area': ['朝阳', '定福庄'], 'url': 'http://bj.58.com/jiadian/24349380911041x.shtml', 'title': '【图】洗衣机，小冰箱，小冰柜，冷饮机 - 朝阳定福庄二手家电 - 北京58同城', 'price': 1500, 'pub_date': '2016.01.14', 'time': 2, '_id': ObjectId('5698f525a98063dbe4e91ca8'), 'look': '-', 'cates': ['北京58同城', '北京二手市场', '北京二手家电', '北京二手洗衣机']}
{'area': ['朝阳', '望京'], 'url': 'http://bj.58.com/diannao/24475337853109x.shtml', 'title': '【图】三星 A5 白色 没有打开过 - 朝阳望京台式机/配件 - 北京58同城', 'price': 1500, 'pub_date': '2015.12.27', 'time': 3, '_id': ObjectId('5698f525a98063dbe7e91ca8'), 'look': '-', 'cates': ['北京58同城', '北京二手市场', '北京二手台式机/配件']}
{'area': ['丰台'], 'url': 'http://bj.58.com/bijibendiannao/24220974748090x.shtml', 'title': '【图】转让自用的电脑 保护的很新，几乎没用 - 丰台笔记本电脑 - 北京58同城', 'price': 0, 'pub_date': '2015.12.31', 'time': 2, '_id': ObjectId('5698f525a98063dbe8e91ca8'), 'look': '-', 'cates': ['北京58同城', '北京二手市场', '北京二手笔记本']}
{'area': ['朝阳', '高碑店'], 'url': 'http://bj.58.com/jiadian/24541664530488x.shtml', 'title': '【图】95成新小冰柜转让 - 朝阳高碑店二手家电 - 北京58同城', 'price':

In [10]:
pipeline = [
    {'$group':{'_id':{'$slice':['$area',1]}}},
    {'$sort':{'_id':-1}},
]

In [31]:
def gen_area():
    pipeline = [
    {'$group':{'_id':{'$slice':['$area',1]}}},
    {'$sort':{'_id':1}},
    ]
    for item in item_info.aggregate(pipeline):
        yield item['_id'][0]
    

In [19]:
pipeline2 = [
    {'$match' : {'area' : {'$all' : ['昌平']}}},
    {'$group' : {'_id' : {'$slice' : ['$cates', 2, 1]}, 'counts' : {'$sum' : 1}}},
    {'$sort' : {'counts': -1}},
    {'$limit' : 3}
    
]

In [20]:
for i in item_info.aggregate(pipeline2):
    print(i)

{'_id': ['北京二手家电'], 'counts': 1247}
{'_id': ['北京二手母婴/儿童用品'], 'counts': 546}
{'_id': ['北京二手文体/户外/乐器'], 'counts': 523}


In [23]:
def gen_data(area):
    pipeline2 = [
        {'$match' : {'area' : {'$all' : [area]}}},
        {'$group' : {'_id' : {'$slice' : ['$cates', 2, 1]}, 'counts' : {'$sum' : 1}}},
        {'$sort' : {'counts': -1}},
        {'$limit' : 3}
    ]
    for icate in item_info.aggregate(pipeline2):
        data = {
            'name' : icate['_id'][0],
            'data' : [icate['counts']],
            'type' : 'column'
        }
        yield data
        


In [32]:
def plot_top3():
    options = {
    'chart'   : {'zoomType':'xy'},
    'title'   : {'text': 'top cate'},
    'subtitle': {'text': 'data chart'},
    'yAxis'   : {'title': {'text': 'total'}}
    }
    for area in gen_area():
        series = [ data for data in gen_data(area)]
        charts.plot(series, options = options, show = 'inline')
        print(area)
        

In [33]:
plot_top3()

不明
东城
丰台
北京周边
大兴
宣武
密云
崇文
平谷
延庆
怀柔
房山
昌平
朝阳
海淀
燕郊
石景山
西城
通州
门头沟
附近
顺义


In [36]:
series = [ data for data in gen_data('顺义')]
options = {
    'chart'   : {'zoomType':'xy'},
    'title'   : {'text': 'top cate'},
    'subtitle': {'text': 'data chart'},
    'yAxis'   : {'title': {'text': 'total'}}
    }
charts.plot(series, options = options, show = 'inline')

In [3]:
cates = '北京二手家电'
area = '顺义'
pipeline3 = [
    {'$match' :{'$and' : [{'look' : {'$nin' : ['-']}},
                         {'cates' : {'$all' : [cates]}},
                         {'area' : area},
                         ]}},
    {'$group' : {'_id' : '$look', 'avg_price':{'$avg':'$price'}}},
    {'$sort' : {'avg_price' : 1}}
]

In [4]:
for ilook in item_info.aggregate(pipeline3):
    print(ilook)

{'_id': '全新', 'avg_price': 621.6363636363636}
{'_id': '8成新', 'avg_price': 638.0}
{'_id': '9成新', 'avg_price': 1070.8333333333333}
{'_id': '95成新', 'avg_price': 1318.0}


In [15]:
def plot_look(cate, area = None):
    if area == None:
        pipeline3 = [
            {'$match' :{'$and' : [{'look' : {'$nin' : ['-']}},
                                 {'cates' : {'$all' : [cates]}},
                                 ]}},
            {'$group' : {'_id' : '$look', 'avg_price':{'$avg':'$price'}}},
            {'$sort' : {'avg_price' : 1}}
        ]
    else:
        pipeline3 = [
            {'$match' :{'$and' : [{'look' : {'$nin' : ['-']}},
                                 {'cates' : {'$all' : [cates]}},
                                 {'area' : area},
                                 ]}},
            {'$group' : {'_id' : '$look', 'avg_price':{'$avg':'$price'}}},
            {'$sort' : {'avg_price' : 1}}
        ]
    
    for item in item_info.aggregate(pipeline3):
        data = {
            'name' : item['_id'],
            'price' : item['avg_price']
        }
        yield data
    
    

In [27]:
data = [i['price'] for i in plot_look(cates)]
xLable = [i['name'] for i in plot_look(cates)]
options = {
    'title': {'text': '新旧-价格'},
    'yAxis'   : {'title': {'text': '价格'}}, 
    'xAxis' : {'categories' : xLable }
}
charts.line(data, show='inline',options=options)


In [24]:
data

[389.47169811320754,
 685.2716388126715,
 759.0126582278481,
 1415.9157894736843,
 3016.755719557196,
 4003.2659974905896]